# Install required libraries

In [ ]:
!pip install -q openai-whisper ffmpeg-python
!pip install thefuzz
!pip install easyocr
!pip install vosk
!pip install difflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s

# Import required libraries

In [ ]:
import whisper
import ffmpeg
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#vosk
import subprocess
import wave
import json
from vosk import Model, KaldiRecognizer, SetLogLevel

import cv2
import easyocr
from thefuzz import fuzz
from difflib import SequenceMatcher

# Helper function

In [ ]:
embd_model = SentenceTransformer('all-MiniLM-L6-v2')
# embd_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# embd_model = SentenceTransformer('Snowflake/snowflake-arctic-embed-l-v2.0')


def format_time(seconds):
    minutes = int(seconds // 60)
    remaining_seconds = round(seconds % 60, 2)

    return minutes, remaining_seconds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Audio

In [ ]:
#get text from video
def transcribe_video(video_path, model_size="small", audio_path='/content/temp_audio.mp3'):

    ffmpeg.input(video_path).output(audio_path, format="mp3", acodec="libmp3lame", ar="16k").run(overwrite_output=True)

    model = whisper.load_model(model_size)

    result = model.transcribe(audio_path, word_timestamps=True)

    return result




#transform sentences to embeddings
def get_embedding(sentence):

    embedding = embd_model.encode(sentence)
    return embedding

#filtering the output of whisper model
def extract_segments(data):

    segments_dict = {}

    for segment in data.get('segments', []):  # Get segments safely
        segment_id = segment['id']
        segments_dict[segment_id] = {
            'start': segment['start'],
            'end': segment['end'],
            'text': segment['text']
        }

    return segments_dict

#creating embedding matrix from video text
def create_embeddings(segments_dict):
    ids = []
    embeddings = []

    ids = list(segments_dict.keys())  # Extract segment IDs
    texts = [segments_dict[seg_id]['text'] for seg_id in ids]  # Extract texts

    embeddings = embd_model.encode(texts, batch_size=32, show_progress_bar=True)

    # for segment_id, data in segments_dict.items():
    #     text = data['text']
    #     embedding = get_embedding(text)

    #     ids.append(segment_id)
    #     embeddings.append(embedding)

    return np.array(ids), np.array(embeddings)

#return 5 most relevant timestamp and corresponding text to user inputted text
def search_similar_segments_audio(user_text, segments_dict, ids, embedding_matrix, top_k=5):

    user_embedding = get_embedding(user_text).reshape(1, -1)

    #compute cosine similarity between user input and stored embeddings
    similarities = cosine_similarity(user_embedding, embedding_matrix)[0]

    #get the indices of the top_k most similar segments
    top_indices = np.argsort(similarities)[::-1][:top_k]

    #retrieve top_k results
    top_results = []
    for idx in top_indices:
        segment_id = ids[idx]
        # print(segments_dict[segment_id])
        segment_text = segments_dict[segment_id]['text']
        similarity_score = similarities[idx]
        start_time = segments_dict[segment_id]['start']
        end_time = segments_dict[segment_id]['end']
        top_results.append((segment_text, similarity_score, start_time, end_time))

    return top_results

In [ ]:
def audio_final_results(video_path,user_input):
    audio_result = transcribe_video(video_path)
    er = extract_segments(audio_result)
    id,em_mat = create_embeddings(er)

    top_results = search_similar_segments_audio(user_input, er, id, em_mat)

    return top_results

In [ ]:
# video_file = "/content/lr.mp4"
# audio_result = transcribe_video(video_file)
# er1 = extract_segments(audio_result)
# print(er1)
# id1,em_mat1 = create_embeddings(er1)

100%|███████████████████████████████████████| 461M/461M [00:21<00:00, 22.5MiB/s]


{0: {'start': np.float64(0.0), 'end': np.float64(6.58), 'text': ' Welcome back. In this video I will discuss reinforcement learning. Reinforcement learning'}, 1: {'start': np.float64(6.58), 'end': np.float64(11.8), 'text': ' is a feedback based machine learning approach. In this case the agent will look into the'}, 2: {'start': np.float64(11.8), 'end': np.float64(17.68), 'text': ' environment. Based on the environment situation he will perform some actions. For each action'}, 3: {'start': np.float64(17.68), 'end': np.float64(23.92), 'text': ' he will be given a feedback in the form of rewards. For example let us assume that he'}, 4: {'start': np.float64(23.92), 'end': np.float64(28.78), 'text': ' has performed some correct action. Then he will be given a positive feedback. If he has'}, 5: {'start': np.float64(28.78), 'end': np.float64(33.54), 'text': ' performed a negative action he will be given a negative feedback in this case or it is'}, 6: {'start': np.float64(33.54), 'end': np.flo

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# user_input = "optimal path"
# top_results1 = search_similar_segments_audio(user_input, er1, id1, em_mat1)

results = audio_final_results("/content/lr.mp4","optimal path")
#

for i in range(len(results)):
    st_min,st_sec = format_time(results[i][2])
    et_min,et_sec = format_time(results[i][3])
    print(f"{i+1}. : {st_min}.{int(st_sec)} - {et_min}.{int(et_sec)} : {results[i][0]}")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

1. : 7.51 - 7.55 :  an optimal path so that the user can go through this particular goal over here.
2. : 7.7 - 7.12 :  the place where the agent is present and if you want to find an optimal path so
3. : 1.24 - 1.29 :  he has to follow an optimal path over here. Let us say that when he is present in this
4. : 6.58 - 7.2 :  you are designing a robot in such cases we can use this reinforcement learning.
5. : 8.30 - 8.34 :  kind of problems rather than going with this particular reinforcement learnings


In [ ]:
print(results)

[(' an optimal path so that the user can go through this particular goal over here.', np.float32(0.7241397), np.float64(471.4), np.float64(475.4)), (' the place where the agent is present and if you want to find an optimal path so', np.float32(0.6369792), np.float64(427.76), np.float64(432.86)), (' he has to follow an optimal path over here. Let us say that when he is present in this', np.float32(0.56353164), np.float64(84.68), np.float64(89.54)), (' you are designing a robot in such cases we can use this reinforcement learning.', np.float32(0.35295552), np.float64(418.26), np.float64(422.16)), (' kind of problems rather than going with this particular reinforcement learnings', np.float32(0.3502155), np.float64(510.48), np.float64(514.56))]


# Video

In [ ]:
def extract_ocr_from_video(video_path, output_json_path="ocr_output_easyocr.json", lang_list=['en'], similarity_threshold=75):
    """
    Extracts text from a video using OCR (EasyOCR), grouping similar consecutive frames.

    Parameters:
        video_path (str): Path to the input video file.
        output_json_path (str): Path to save the OCR results as JSON.
        lang_list (list): List of languages for EasyOCR.
        similarity_threshold (int): Similarity threshold (0-100) to group consecutive similar texts.

    Returns:
        list: List of dictionaries containing 'text', 'start_time', and 'end_time' keys.
    """
    # Initialize EasyOCR reader
    reader = easyocr.Reader(lang_list)

    # Load video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps)
    video_duration = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

    ocr_results = []
    prev_text = None
    start_time = None

    for sec in range(video_duration):
        cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        result = reader.readtext(gray_frame, detail=0)
        text = " ".join(result).strip() if result else "No text detected"

        if prev_text is not None and fuzz.ratio(text, prev_text) >= similarity_threshold:
            end_time = sec
        else:
            if prev_text is not None:
                ocr_results.append({
                    "text": prev_text,
                    "start_time": start_time,
                    "end_time": end_time
                })
            prev_text = text
            start_time = sec
            end_time = sec

    if prev_text is not None:
        ocr_results.append({
            "text": prev_text,
            "start_time": start_time,
            "end_time": end_time
        })

    cap.release()

    with open(output_json_path, "w") as f:
        json.dump(ocr_results, f, indent=4)

    # print(json.dumps(ocr_results, indent=4))
    return ocr_results


In [ ]:
# def search_similar_segments_video(query, ocr_data, top_k=5):
#     results = []

#     for segment in ocr_data:
#         if query.lower() in segment["text"].lower():
#             results.append((segment["text"], segment["start_time"], segment["end_time"]))

#     results = sorted(results, key=lambda x: x[1])  # Sort by start time
#     return results[:top_k]  # Return top 5 matches

In [ ]:
def process_ocr_results(ocr_results,SIMILARITY_THRESHOLD = 85):
    merged_results = []
    if not ocr_results:
        print('hi')
        return merged_results

    current_group = ocr_results[0].copy()
    for i in range(1, len(ocr_results)):
        current_text = ocr_results[i]['text']
        similarity = fuzz.ratio(current_group['text'], current_text)

        if similarity >= SIMILARITY_THRESHOLD:
            current_group['end_time'] = ocr_results[i]['end_time']
        else:
            merged_results.append(current_group)
            current_group = ocr_results[i].copy()

    merged_results.append(current_group)
    return merged_results



def remove_repeated_sentence(merged_results,SIMILARITY_THRESHOLD2 = 75):
    # Step 2: Remove repeated sentences across all merged segments
    seen_sentences = []
    final_results = []
    for segment in merged_results:
        sentences = [s.strip() for s in segment['text'].split('. ') if s.strip()]
        unique_sentences = []
        for sentence in sentences:
            is_new = True
            for seen in seen_sentences:
                if fuzz.ratio(sentence, seen) >= SIMILARITY_THRESHOLD2:
                    is_new = False
                    break
            if is_new:
                unique_sentences.append(sentence)
                seen_sentences.append(sentence)

        if unique_sentences:
            final_results.append({
                "text": ". ".join(unique_sentences),
                "start_time": segment["start_time"],
                "end_time": segment["end_time"]
            })

    return final_results

In [ ]:
def video_final_results(video_path,user_input,top_k=5):
    ocr_results = extract_ocr_from_video(video_path)
    merged_results = process_ocr_results(ocr_results)


    clean_ocr = remove_repeated_sentence(merged_results)

    # Prepare embeddings for OCR
    ocr_texts = [seg['text'] for seg in clean_ocr]
    ocr_embeddings = embd_model.encode(ocr_texts, batch_size=32, show_progress_bar=False)
    user_embedding = get_embedding(user_input).reshape(1, -1)
    ocr_similarities = cosine_similarity(user_embedding, ocr_embeddings)[0]

    top_ocr_indices = np.argsort(ocr_similarities)[::-1][:top_k]
    top_ocr = []
    for idx in top_ocr_indices:
        seg = clean_ocr[idx]
        top_ocr.append((seg['text'], ocr_similarities[idx], seg['start_time'], seg['end_time']))

    return top_ocr

In [ ]:
ocr_res = video_final_results('/content/lr.mp4','optimal path')

for i in range(len(ocr_res)):
    st_min,st_sec = format_time(ocr_res[i][2])
    et_min,et_sec = format_time(ocr_res[i][3])
    print(f"{i+1}. : {st_min}.{int(st_sec)} - {et_min}.{int(et_sec)} : {ocr_res[i][0]}")

[
    {
        "text": "Reinforcement Learning Environment  Introduction State Action Agent Reward Like, Share and Subscribe to Mahesh Huddar Visit: vtupulse.com",
        "start_time": 0,
        "end_time": 4
    },
    {
        "text": "Introduction to Reinforcement Learning Like, Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com",
        "start_time": 5,
        "end_time": 5
    },
    {
        "text": "Introduction to Reinforcement Learning Reinforcement Learning feedback-based Machine learning Approach here agent learns to which actions to perform by the environment and the results actions. Like, Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com looking",
        "start_time": 6,
        "end_time": 39
    },
    {
        "text": "Introduction to Reinforcement Learning Reinforcement Learning feedback-based Machine learning Approach here agent Environment learns to which actions to perform by looking the environment and the results actions. For each correct act

# Audio + Video

In [ ]:
def top_combined_results(video_path, user_input, top_k_each=5, final_top_k=5):
    #Get top 5 from audio
    audio_top_results = audio_final_results(video_path, user_input)

    #Get top 5 from OCR
    ocr_top_results = video_final_results(video_path, user_input)

    #Combine results
    combined_segments = audio_top_results + ocr_top_results

    #Recompute embeddings of all 10 for fair comparison
    texts = [seg[0] for seg in combined_segments]
    embeddings = embd_model.encode(texts, batch_size=10, show_progress_bar=False)
    user_embedding = get_embedding(user_input).reshape(1, -1)

    similarities = cosine_similarity(user_embedding, embeddings)[0]

    #Get final top 5 across audio + OCR
    top_indices = np.argsort(similarities)[::-1][:final_top_k]
    final_top_results = []
    for idx in top_indices:
        text, _, start, end = combined_segments[idx]
        final_top_results.append((text, similarities[idx], start, end))

    return final_top_results


In [ ]:
final_results = top_combined_results("/content/lr.mp4", "like and subscribe")

for i, (text, score, start, end) in enumerate(final_results, 1):
    st_min, st_sec = format_time(start)
    et_min, et_sec = format_time(end)
    print(f"{i}. : {st_min}.{int(st_sec)} - {et_min}.{int(et_sec)} : {text}")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

1. : 0.5 - 0.5 : Introduction to Reinforcement Learning Like, Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
2. : 8.48 - 8.52 :  with your friends. Press the subscribe button for more videos. Press the bell
3. : 4.6 - 4.29 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
4. : 0.40 - 2.24 : For each correct action; the agent  gets State positive feedback, and for each incorrect Action Agent Reward action, the agent gets negative feedback or penalty: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
5. : 4.30 - 5.9 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and the frequency of a specific behaviour: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com


In [ ]:
print(final_results)

[('Introduction to Reinforcement Learning Like, Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com', np.float32(0.45927805), 5, 5), (' with your friends. Press the subscribe button for more videos. Press the bell', np.float32(0.45766962), np.float64(528.56), np.float64(532.08)), ('Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com', np.float32(0.40449744), 246, 269), ('For each correct action; the agent  gets State positive feedback, and for each incorrect Action Agent Reward action, the agent gets negative feedback or penalty: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com', np.float32(0.3474167), 40, 144), ('Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and the frequency 

# Class

In [ ]:
class VideoProcessor:
    def __init__(self, video_path):
        self.video_path = video_path
        self.segments_dict = None
        self.ids = None
        self.embedding_matrix = None
        self.clean_ocr = None
        self.ocr_embeddings = None

        self.embd_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.__preprocess()


    def format_time(self,seconds):
        minutes = int(seconds // 60)
        remaining_seconds = round(seconds % 60, 2)

        return minutes, remaining_seconds

    #get text from video
    def __transcribe_video(self, model_size="small", audio_path='/content/temp_audio.mp3'):

        ffmpeg.input(self.video_path).output(audio_path, format="mp3", acodec="libmp3lame", ar="16k").run(overwrite_output=True)
        model = whisper.load_model(model_size)
        result = model.transcribe(audio_path, word_timestamps=True)

        return result

    #filtering the output of whisper model
    def __extract_segments(self,data):

        segments_dict = {}

        for segment in data.get('segments', []):  # Get segments safely
            segment_id = segment['id']
            segments_dict[segment_id] = {
                'start': segment['start'],
                'end': segment['end'],
                'text': segment['text']
            }

        return segments_dict

    #creating embedding matrix from video text
    def __create_embeddings(self,segments_dict):
        ids = []
        embeddings = []

        ids = list(segments_dict.keys())  # Extract segment IDs
        texts = [segments_dict[seg_id]['text'] for seg_id in ids]  # Extract texts

        embeddings = self.embd_model.encode(texts, batch_size=32, show_progress_bar=True)

        return np.array(ids), np.array(embeddings)



    def __process_ocr_results(self,ocr_results,SIMILARITY_THRESHOLD = 85):
        merged_results = []
        if not ocr_results:
            print('hi')
            return merged_results

        current_group = ocr_results[0].copy()
        for i in range(1, len(ocr_results)):
            current_text = ocr_results[i]['text']
            similarity = fuzz.ratio(current_group['text'], current_text)

            if similarity >= SIMILARITY_THRESHOLD:
                current_group['end_time'] = ocr_results[i]['end_time']
            else:
                merged_results.append(current_group)
                current_group = ocr_results[i].copy()

        merged_results.append(current_group)
        return merged_results



    def __remove_repeated_sentence(self,merged_results,SIMILARITY_THRESHOLD2 = 75):
        # Step 2: Remove repeated sentences across all merged segments
        seen_sentences = []
        final_results = []
        for segment in merged_results:
            sentences = [s.strip() for s in segment['text'].split('. ') if s.strip()]
            unique_sentences = []
            for sentence in sentences:
                is_new = True
                for seen in seen_sentences:
                    if fuzz.ratio(sentence, seen) >= SIMILARITY_THRESHOLD2:
                        is_new = False
                        break
                if is_new:
                    unique_sentences.append(sentence)
                    seen_sentences.append(sentence)

            if unique_sentences:
                final_results.append({
                    "text": ". ".join(unique_sentences),
                    "start_time": segment["start_time"],
                    "end_time": segment["end_time"]
                })

        return final_results


    def __extract_ocr_from_video(self, lang_list=['en'], similarity_threshold=75):

        # Initialize EasyOCR reader
        reader = easyocr.Reader(lang_list)

        # Load video
        cap = cv2.VideoCapture(self.video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps)
        video_duration = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps)

        ocr_results = []
        prev_text = None
        start_time = None

        for sec in range(video_duration):
            cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
            ret, frame = cap.read()
            if not ret:
                break

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            result = reader.readtext(gray_frame, detail=0)
            text = " ".join(result).strip() if result else "No text detected"

            if prev_text is not None and fuzz.ratio(text, prev_text) >= similarity_threshold:
                end_time = sec
            else:
                if prev_text is not None:
                    ocr_results.append({
                        "text": prev_text,
                        "start_time": start_time,
                        "end_time": end_time
                    })
                prev_text = text
                start_time = sec
                end_time = sec

        if prev_text is not None:
            ocr_results.append({
                "text": prev_text,
                "start_time": start_time,
                "end_time": end_time
            })

        cap.release()

        return ocr_results


    def __preprocess(self):
        # Audio processing
        audio_result = self.__transcribe_video()
        self.segments_dict = self.__extract_segments(audio_result)
        self.ids, self.embedding_matrix = self.__create_embeddings(self.segments_dict)

        # OCR processing
        ocr_results = self.__extract_ocr_from_video()
        merged_results = self.__process_ocr_results(ocr_results)
        self.clean_ocr = self.__remove_repeated_sentence(merged_results)
        ocr_texts = [seg['text'] for seg in self.clean_ocr]
        self.ocr_embeddings = self.embd_model.encode(ocr_texts, batch_size=32, show_progress_bar=False)

    def query(self, user_input, top_k_each=5, final_top_k=5):
        user_embedding = self.embd_model.encode(user_input).reshape(1, -1)

        # Audio
        similarities_audio = cosine_similarity(user_embedding, self.embedding_matrix)[0]
        top_audio_indices = np.argsort(similarities_audio)[::-1][:top_k_each]
        top_audio = [(self.segments_dict[self.ids[idx]]['text'], similarities_audio[idx],
                      self.segments_dict[self.ids[idx]]['start'], self.segments_dict[self.ids[idx]]['end'])
                     for idx in top_audio_indices]

        # OCR
        similarities_ocr = cosine_similarity(user_embedding, self.ocr_embeddings)[0]
        top_ocr_indices = np.argsort(similarities_ocr)[::-1][:top_k_each]
        top_ocr = [(self.clean_ocr[idx]['text'], similarities_ocr[idx],
                    self.clean_ocr[idx]['start_time'], self.clean_ocr[idx]['end_time'])
                   for idx in top_ocr_indices]

        # Combine and rerank
        combined = top_audio + top_ocr
        texts = [item[0] for item in combined]
        embeddings = self.embd_model.encode(texts, batch_size=10, show_progress_bar=False)
        similarities = cosine_similarity(user_embedding, embeddings)[0]
        top_indices = np.argsort(similarities)[::-1][:final_top_k]

        return [(texts[i], similarities[i], combined[i][2], combined[i][3]) for i in top_indices]


In [ ]:
temp = VideoProcessor('/content/lr.mp4')

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
final_results = temp.query('like and subscribe')

for i, (text, score, start, end) in enumerate(final_results, 1):
    st_min, st_sec = temp.format_time(start)
    et_min, et_sec = temp.format_time(end)
    print(f"{i}. : {st_min}.{int(st_sec)} - {et_min}.{int(et_sec)} : {text}")

1. : 0.5 - 0.5 : Introduction to Reinforcement Learning Like, Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
2. : 8.48 - 8.52 :  with your friends. Press the subscribe button for more videos. Press the bell
3. : 4.6 - 4.29 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
4. : 0.40 - 2.24 : For each correct action; the agent  gets State positive feedback, and for each incorrect Action Agent Reward action, the agent gets negative feedback or penalty: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
5. : 4.30 - 5.9 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and the frequency of a specific behaviour: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com


In [ ]:
final_results = temp.query('types of reinforcement learning')

for i, (text, score, start, end) in enumerate(final_results, 1):
    st_min, st_sec = temp.format_time(start)
    et_min, et_sec = temp.format_time(end)
    print(f"{i}. : {st_min}.{int(st_sec)} - {et_min}.{int(et_sec)} : {text}")

1. : 3.27 - 3.32 :  Reinforcement learning can be used to solve specific type of problems where the decision
2. : 3.55 - 4.0 :  the reinforcement learning in that case. Some of the examples where we can use reinforcement
3. : 4.30 - 5.9 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and the frequency of a specific behaviour: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
4. : 4.6 - 4.29 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com
5. : 5.10 - 5.17 : Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and t

In [ ]:
final_results

[(' Reinforcement learning can be used to solve specific type of problems where the decision',
  np.float32(0.77989376),
  np.float64(207.14),
  np.float64(212.14)),
 (' the reinforcement learning in that case. Some of the examples where we can use reinforcement',
  np.float32(0.73755383),
  np.float64(235.74),
  np.float64(240.42)),
 ('Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Positive reinforcement learning is recurrence of behaviour due to positive rewards: Rewards increase strength and the frequency of a specific behaviour: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com',
  np.float32(0.70653284),
  270,
  309),
 ('Introduction to Reinforcement Learning There are two types of reinforcement learning - positive and negative: Like; Share and Subscribe t0 Mahesh Huddar Visit: vtupulse com',
  np.float32(0.70283806),
  246,
  269),
 ('Introduction to Reinforcement Learning There are two types of reinforc